In [1]:
import torch
torch.__version__

'2.0.1'

In [2]:
torch.backends.mps.is_available()

True

In [3]:
mps_device = torch.device('mps')
x = torch.ones(5, device=mps_device)
x

tensor([1., 1., 1., 1., 1.], device='mps:0')